# Fitting and optimization

We will use scipy (https://scipy.org/) for fitting and optimization, in particular scipy.optimize (https://docs.scipy.org/doc/scipy/reference/optimize.html).

## Physical constants (unrelated aside)

In [ ]:
import scipy.constants as const

In [ ]:
# convert temperatures:
const.convert_temperature(100, old_scale='C', new_scale='K')

In [ ]:
# more constants (including units and errors)!

for k, v in const.physical_constants.items():
    print(k, ':', v)

In [ ]:
const.h

In [ ]:
val, unit, uncertainty = const.physical_constants['Planck constant']
val, unit, uncertainty

## Fitting a scatter plot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

In [ ]:
xdata = np.array([ -10.0, -9.0, -8.0, -7.0, -6.0, -5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0])
ydata = np.array([1.2, 4.2, 5.2, 8.3, 10.6, 11.7, 13.5, 14.5, 15.7, 16.1, 16.6, 16.0, 15.4, 14.4, 14.2, 12.7, 10.3, 8.6, 6.1, 4.9, 2.1])

In [ ]:
plt.scatter(xdata, ydata)

#### Let's try and fit a Gaussian to our data
Recall that a Gaussian function has three parameters: *a* controls the height, *b* the position of the peak, and *c* the width of the function.

In [ ]:
# Define the Gaussian function
def gaussian(x, a, b, c):
    y = a * np.exp( -(x - b)** 2 / 2*c**2)
    return y

# We use curve_fit (https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html), which has required inputs: 
# 1) the fit function f 
# 2) an array of data points in x 
# 3) an array of data points in y
# curve_fit returns as default optimal values for f for a least-squares fit, and the covariance matrix, as an array and 2D array, respectively
parameters, covariance = opt.curve_fit(gaussian, xdata, ydata)
print(parameters)
fitted_a, fitted_b, fitted_c = parameters
print(fitted_a, fitted_b, fitted_c)

#### Let's overplot the fit function with the optimal values over the data 

In [ ]:
fit_y = gaussian(xdata, fitted_a, fitted_b, fitted_c)
plt.scatter(xdata, ydata, label='data')
plt.plot(xdata, fit_y, '-', label='Gaussian fit', color='r', lw=3,alpha=0.7)
plt.legend()
plt.show()

#### We can get the uncertainties from the covariance matrix:

In [ ]:
print(covariance)
sigma_a = covariance[0,0] ** 0.5
sigma_b = covariance[1,1] ** 0.5
sigma_c = covariance[2,2] ** 0.5

print(f"\nsigma_a={sigma_a:.1e}, sigma_b={sigma_b:.1e}, sigma_c={sigma_c:.1e}")

#### We can also plot the correlation matrix
The @ operator is used for matrix multiplication.

In [ ]:
def cov2cor(cov):
    '''Convert the covariance matrix to the correlation matrix'''
    D = np.diag(1 / np.sqrt(np.diag(cov)))
    return D @ cov @ D

In [ ]:
correlation_matrix = cov2cor(covariance)

plt.matshow(correlation_matrix, vmin=-1, vmax=1, cmap='RdBu_r')
plt.colorbar(shrink=0.8);

correlation_matrix

#### Now fit a cosine

In [ ]:
def cos_func(x, D, E):
    y = D*np.cos(E*x)
    return y

parameters, covariance = opt.curve_fit(cos_func, xdata, ydata)
fitted_d = parameters[0]
fitted_e = parameters[1]

fit_cosine = cos_func(xdata, fitted_d, fitted_e)

plt.plot(xdata, ydata, 'o', label='data')
plt.plot(xdata, fit_cosine, '-', label='cosine fit')

### Fit with an initial guess
You can make an initial guess for the parameters of the function, passing curve_fit() a list or array of guesses for the function parameters. This is an optional parameter. 

In [ ]:
guess = [16, 0.1]
parameters, covariance = opt.curve_fit(cos_func, xdata, ydata, p0=guess)
fitted_d, fitted_e = parameters

fit_cosine = cos_func(xdata, fitted_d, fitted_e)

plt.plot(xdata, ydata, 'o', label='data')
plt.plot(xdata, fit_cosine, '-', label='cosine fit', color='r', lw=3,alpha=0.7)

print(parameters)

### Fitting data with error bars

#### First let's make some error bars in y

In [ ]:
y_err = 2./np.sqrt(ydata)
plt.errorbar(xdata, ydata, yerr=y_err, fmt='o')

An array of uncertainties in y is an optional parameter for `curve_fit()`. This can be a 1D or 2D array (covariance matrix of errors in ydata for the latter). In general, `absolute_sigma` should be `True`, so that the covariance is calculated based on the absolute, rather than relative, values of the uncertainties. 

In [ ]:
parameters, covariance = opt.curve_fit(gaussian, xdata, ydata, sigma=y_err,absolute_sigma=True)
print(parameters)
fitted_a, fitted_b, fitted_c = parameters
print(fitted_a, fitted_b, fitted_c)

In [ ]:
fit_y = gaussian(xdata, fitted_a, fitted_b, fitted_c)
plt.errorbar(xdata, ydata, yerr=y_err, fmt='o');
plt.plot(xdata, fit_y, '-', label='Gaussian fit', color='r', lw=3,alpha=0.7)
plt.legend()
plt.show()

In [ ]:
# Calculate the pull distribution
pull = (fit_y - ydata)/y_err

plt.plot(xdata, pull, 'o', label='Pull')
plt.legend()
plt.show()

In [ ]:
plt.hist(pull, bins=15);

This doesn't look Gaussian or to be centered on zero, but we aren't surprised, because our error bars were not Gaussian.

In [ ]:
y_err = np.random.normal(1, 0.02, size=len(xdata))
plt.errorbar(xdata, ydata, yerr=y_err, fmt='o');

In [ ]:
fit_y = gaussian(xdata, fitted_a, fitted_b, fitted_c)
plt.errorbar(xdata, ydata, yerr=y_err, fmt='o');
plt.plot(xdata, fit_y, '-', label='Gaussian fit', color='r', lw=3,alpha=0.7)
plt.legend()
plt.show()

In [ ]:
# Calculate the pull distribution
pull = (fit_y - ydata)/y_err

plt.plot(xdata, pull, 'o', label='Pull')
plt.legend()
plt.show()

In [ ]:
plt.hist(pull, bins=15);

Hard to tell if this is Gaussian/centered at zero with so few entries, but looks more likely than the last pull distribution.

## Fitting Histograms

In [ ]:
# Generate random data:
mean = 0
sigma = 1
x = np.random.normal(0, 1, 1000)
plt.hist(x, bins=25);

hist, bin_edges = np.histogram(x)
# normalize to unit area
hist=hist/sum(hist)

In [ ]:
# Extract histogram values into numpy arrays
n = len(hist)
x_hist=np.zeros((n),dtype=float) 
for i in range(n):
    x_hist[i]=(bin_edges[i+1]+bin_edges[i])/2
    
y_hist=hist

print(x_hist)
print(y_hist)

Since we have extracted the x and y data into arrays, we can use `curve_fit` very similarly to when fitting a scatter plot, passing the fit function, and arrays of data in x and y.

In [ ]:
# Gaussian least-square fitting process
# no initial guess
#param_optimised,param_covariance_matrix = opt.curve_fit(gaussian,x_hist,y_hist)
# with an initial guess
param_optimised,param_covariance_matrix = opt.curve_fit(gaussian,x_hist,y_hist,p0=[16, 0.1, 0.9])

In [ ]:
fig = plt.figure()
x_hist_2=np.linspace(np.min(x_hist),np.max(x_hist),1000)
plt.plot(x_hist_2,gaussian(x_hist_2,*param_optimised),'r',label='Gaussian fit')
plt.legend()

#Normalise the histogram values
weights = np.ones_like(x) / len(x)
plt.hist(x, weights=weights)

#setting the y-axis label
plt.ylabel("Probability")

## Minimization

#### Define the function to minimize and plot it.

In [ ]:
f = lambda x : (x - 2) ** 2 * (x + 2) ** 2 - x

x = np.linspace(-3,3,500)
plt.plot(x, f(x))
plt.show()

In [ ]:
# We use minimize_scalar to find the minimum (https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize_scalar.html). 
# Only required argument is the function to be minimized.
sol = opt.minimize_scalar(f)

print(sol)

plt.plot(x, f(x))
plt.scatter([sol.x], [sol.fun], marker='x', s=100, c='green',label='global minimum')
plt.legend()
plt.show()

In [ ]:
# To select a different local minimum: use bounded minimization

sol_local = opt.minimize_scalar(f, method='bounded', bounds=[-3,0])

print(sol_local)

plt.plot(x, f(x))
plt.scatter([sol_local.x], [sol_local.fun],marker='x', s=100, c='red', label='local minimum')
plt.legend()
plt.show()

### Some more fun with plotting: Visualizing your multi-dimensional functions

In [ ]:
def egg_crate(x,y):
    return x ** 2 + y ** 2 + 25 * (np.sin(x) ** 2 + np.sin(y) ** 2)

In [ ]:
x = np.arange(-5, 5, 0.1)
y = np.arange(-5, 5, 0.1)
X, Y = np.meshgrid(x, y)
Z = egg_crate(X,Y)
print (Z.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))

ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, cmap='RdBu')

In [ ]:
def plot_heat(X=X, Y=Y, Z=Z):
    fig, ax = plt.subplots(figsize=(7,6))
    c = ax.pcolormesh(X, Y, Z, cmap='RdBu', vmin=np.min(Z), vmax=np.max(Z))
    fig.colorbar(c, ax=ax)
    plt.axis([np.min(X), np.max(X), np.min(Y), np.max(Y)])
    
plot_heat()
plt.show()